# 3 Pointers Made against game_details.csv

### Import packages

In [25]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [26]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [27]:
df = pd.read_csv('backend/data/details/game_details.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = shooting_df[['date', 'visitor', 'home', 'team', '3p']]

### Basic exploration

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504319 entries, 0 to 504318
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        504319 non-null  object 
 1   visitor     504319 non-null  object 
 2   home        504319 non-null  object 
 3   team        504319 non-null  int64  
 4   starter     504277 non-null  float64
 5   player      503917 non-null  object 
 6   mp          503917 non-null  object 
 7   fg          503917 non-null  float64
 8   fga         503917 non-null  float64
 9   fg_perc     482544 non-null  float64
 10  3p          503917 non-null  float64
 11  3pa         503917 non-null  float64
 12  3p_perc     353182 non-null  float64
 13  ft          503917 non-null  float64
 14  fta         503917 non-null  float64
 15  ft_perc     327033 non-null  float64
 16  orb         503917 non-null  float64
 17  drb         503917 non-null  float64
 18  trb         503917 non-null  float64
 19  as

In [29]:
df.sample(5)

,date,visitor,home,team,starter,player,mp,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus
89816,"Thu, Apr 2, 2009",Milwaukee Bucks,Philadelphia 76ers,1,0.0,Kareem Rush,0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119640,"Mon, Mar 22, 2010",Memphis Grizzlies,Sacramento Kings,0,1.0,Mike Conley,37:38,5.0,13.0,0.385,2.0,4.0,0.500,0.0,0.0,NaN,0.0,3.0,3.0,7.0,2.0,0.0,1.0,1.0,12.0,31.0
433304,"Fri, Jan 10, 2020",Orlando Magic,Phoenix Suns,0,1.0,Wes Iwundu,27:58,1.0,7.0,0.143,0.0,1.0,0.000,2.0,2.0,1.0,2.0,1.0,3.0,1.0,0.0,1.0,1.0,5.0,4.0,-4.0
95909,"Wed, Nov 4, 2009",Miami Heat,Washington Wizards,0,1.0,Jermaine O'Neal,32:52,2.0,7.0,0.286,0.0,0.0,NaN,0.0,0.0,NaN,2.0,5.0,7.0,2.0,0.0,1.0,2.0,3.0,4.0,17.0
150540,"Sat, Mar 19, 2011",Cleveland Cavaliers,Los Angeles Clippers,0,0.0,Daniel Gibson,27:34,4.0,12.0,0.333,2.0,9.0,0.222,0.0,0.0,NaN,0.0,6.0,6.0,3.0,2.0,0.0,1.0,2.0,10.0,-10.0


In [30]:
def convert_mp(mp):
    if mp == '0' or mp == 0:
        return 0
    else:
        mins = int(mp.split(':')[0])
        secs = int(mp.split(':')[1]) / 60
        return mins + secs

In [31]:
# Fill NaN
df = df.fillna(0)

# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Convert 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Convert 'minutes played' to float
df['mp'] = df['mp'].apply(lambda x: convert_mp(x))

# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504319 entries, 0 to 504318
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        504319 non-null  datetime64[ns]
 1   visitor     504319 non-null  object        
 2   home        504319 non-null  object        
 3   team        504319 non-null  object        
 4   starter     504319 non-null  float64       
 5   player      504319 non-null  object        
 6   mp          504319 non-null  float64       
 7   fg          504319 non-null  float64       
 8   fga         504319 non-null  float64       
 9   fg_perc     504319 non-null  float64       
 10  3p          504319 non-null  float64       
 11  3pa         504319 non-null  float64       
 12  3p_perc     504319 non-null  float64       
 13  ft          504319 non-null  float64       
 14  fta         504319 non-null  float64       
 15  ft_perc     504319 non-null  float64       
 16  or

In [32]:
# Team total stats
teams_df = df.groupby(['date', 'visitor', 'home', 'team']).sum().reset_index()

In [33]:
# Rename target variable
shooting_df = shooting_df.rename({'3p': 'target'}, axis=1)

# Convert 'date' column to Date object
shooting_df['date'] = pd.to_datetime(shooting_df['date'])

# Convert 'team' column to Team Name
shooting_df['team'] = np.where(shooting_df['team'], shooting_df['home'], shooting_df['visitor'])

In [34]:
# Starters total stats
starters_df = df[df['starter'] == 1].groupby(['date', 'visitor', 'home', 'team']).sum()
starters_df = starters_df[stats]
starters_df = starters_df.reset_index()

# Merge dataframes to have target variable
starters_df = pd.merge(starters_df, shooting_df, 
                       left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
                       how='left')

In [35]:
# Bench total stats
bench_df = df[df['starter'] == 0].groupby(['date', 'visitor', 'home', 'team']).sum()
bench_df = bench_df[stats]
bench_df = bench_df.reset_index()

# Merge dataframes to have target variable
bench_df = pd.merge(bench_df, shooting_df, 
                    left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
                    how='left')

# Dataframe of team's last 15 performances

In [36]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = teams_df[teams_df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

teams_df['dates'] = teams_df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
teams_df['date_1'], teams_df['date_2'] = teams_df['dates'].apply(lambda x: x[0]), teams_df['dates'].apply(lambda x: x[1])
teams_df['date_3'], teams_df['date_4'] = teams_df['dates'].apply(lambda x: x[2]), teams_df['dates'].apply(lambda x: x[3])
teams_df['date_5'], teams_df['date_6'] = teams_df['dates'].apply(lambda x: x[4]), teams_df['dates'].apply(lambda x: x[5])
teams_df['date_7'], teams_df['date_8'] = teams_df['dates'].apply(lambda x: x[6]), teams_df['dates'].apply(lambda x: x[7])
teams_df['date_9'], teams_df['date_10'] = teams_df['dates'].apply(lambda x: x[8]), teams_df['dates'].apply(lambda x: x[9])
teams_df['date_11'], teams_df['date_12'] = teams_df['dates'].apply(lambda x: x[10]), teams_df['dates'].apply(lambda x: x[11])
teams_df['date_13'], teams_df['date_14'] = teams_df['dates'].apply(lambda x: x[12]), teams_df['dates'].apply(lambda x: x[13])
teams_df['date_15'] = teams_df['dates'].apply(lambda x: x[14])

In [37]:
# Merge in opponents (see team defensive stats)
starters_df = pd.merge(
    starters_df, 
    starters_df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

starters_df = starters_df[starters_df['team'] != starters_df['team_opp']]

bench_df = pd.merge(
    bench_df, 
    bench_df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

bench_df = bench_df[bench_df['team'] != bench_df['team_opp']]

In [38]:
# Keep date columns in teams
cols = [col for col in teams_df.columns
        if ('date_' in col) or \
        (col in ['date', 'visitor', 'home', 'team'])]
teams_df = teams_df[cols]

# Merge dates with starters
starters_df = pd.merge(starters_df, teams_df, 
                       left_on=['date', 'visitor', 'home', 'team'], 
                       right_on=['date', 'visitor', 'home', 'team'],
                       how='left')

# Merge dates with bench
bench_df = pd.merge(bench_df, teams_df, 
                    left_on=['date', 'visitor', 'home', 'team'], 
                    right_on=['date', 'visitor', 'home', 'team'],
                    how='left')

In [39]:
starters_df.head()

,date,visitor,home,team,fg,fga,3p,3pa,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,mp,target,team_opp,fg_opp,fga_opp,3p_opp,3pa_opp,ft_opp,fta_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,plus_minus_opp,mp_opp,target_opp,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15
0,2006-10-31,Chicago Bulls,Miami Heat,Chicago Bulls,18.0,43.0,3.0,7.0,14.0,18.0,7.0,14.0,21.0,8.0,5.0,2.0,7.0,12.0,53.0,66.0,128.550000,7.0,Miami Heat,20.0,48.0,3.0,13.0,11.0,19.0,4.0,18.0,22.0,9.0,4.0,3.0,14.0,14.0,54.0,-95.0,156.583333,3.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2006-10-31,Chicago Bulls,Miami Heat,Miami Heat,20.0,48.0,3.0,13.0,11.0,19.0,4.0,18.0,22.0,9.0,4.0,3.0,14.0,14.0,54.0,-95.0,156.583333,3.0,Chicago Bulls,18.0,43.0,3.0,7.0,14.0,18.0,7.0,14.0,21.0,8.0,5.0,2.0,7.0,12.0,53.0,66.0,128.550000,7.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2006-10-31,Phoenix Suns,Los Angeles Lakers,Los Angeles Lakers,26.0,51.0,5.0,10.0,15.0,20.0,7.0,26.0,33.0,24.0,7.0,0.0,13.0,10.0,72.0,-5.0,157.600000,6.0,Phoenix Suns,24.0,49.0,6.0,18.0,5.0,5.0,4.0,20.0,24.0,22.0,1.0,4.0,11.0,13.0,59.0,-11.0,159.483333,13.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2006-10-31,Phoenix Suns,Los Angeles Lakers,Phoenix Suns,24.0,49.0,6.0,18.0,5.0,5.0,4.0,20.0,24.0,22.0,1.0,4.0,11.0,13.0,59.0,-11.0,159.483333,13.0,Los Angeles Lakers,26.0,51.0,5.0,10.0,15.0,20.0,7.0,26.0,33.0,24.0,7.0,0.0,13.0,10.0,72.0,-5.0,157.600000,6.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,2006-11-01,Atlanta Hawks,Philadelphia 76ers,Atlanta Hawks,15.0,49.0,3.0,12.0,15.0,18.0,8.0,19.0,27.0,7.0,7.0,4.0,15.0,16.0,48.0,-83.0,164.683333,4.0,Philadelphia 76ers,24.0,56.0,3.0,5.0,21.0,23.0,13.0,21.0,34.0,17.0,7.0,5.0,14.0,12.0,72.0,78.0,168.800000,3.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [40]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

# Starters Analysis

In [41]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = starters_df.copy()
X = starters_df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

### Last 15 Performances (Unweighted)

In [42]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']

last_15 = last_15_games.copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15[stat + '_trend'] = \
                            last_15[stat + '_trend'] + \
                            z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:10])

last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

# Calculate percentages
last_15['fg_perc'] = last_15['fg'] / last_15['fga']
last_15['fg_perc_opp'] = last_15['fg_opp'] / last_15['fga_opp']

last_15['3p_perc'] = last_15['3p'] / last_15['3pa']
last_15['3p_perc_opp'] = last_15['3p_opp'] / last_15['3pa_opp']

last_15['ft_perc'] = last_15['ft'] / last_15['fta']
last_15['ft_perc_opp'] = last_15['ft_opp'] / last_15['fta_opp']

# Calculate advanced stats
last_15['ts_perc'] = last_15['pts'] / (2 * (last_15['fga'] + .44 * last_15['fta']))
last_15['ts_perc_opp'] = last_15['pts_opp'] / (2 * (last_15['fga_opp'] + .44 * last_15['fta_opp']))

last_15['efg_perc'] = (last_15['fg'] + (.5 * last_15['3p'])) / last_15['fga']
last_15['efg_perc_opp'] = (last_15['fg_opp'] + (.5 * last_15['3p_opp'])) / last_15['fga_opp']

last_15['3par'] = last_15['3pa'] / last_15['fga']
last_15['3par_opp'] = last_15['3pa_opp'] / last_15['fga_opp']

last_15['ftr'] = last_15['fta'] / last_15['fga']
last_15['ftr_opp'] = last_15['fta_opp'] / last_15['fga_opp']

last_15['orb_perc'] = last_15['orb'] / (last_15['orb'] + last_15['drb_opp'])
last_15['orb_perc_opp'] = last_15['orb_opp'] / (last_15['orb_opp'] + last_15['drb'])

last_15['drb_perc'] = last_15['drb'] / (last_15['drb'] + last_15['orb_opp'])
last_15['drb_perc_opp'] = last_15['drb_opp'] / (last_15['drb_opp'] + last_15['orb'])

last_15['trb_perc'] = last_15['trb'] / (last_15['trb'] + last_15['trb_opp'])
last_15['trb_perc_opp'] = last_15['trb_opp'] / (last_15['trb_opp'] + last_15['trb'])

last_15['ast_perc'] = last_15['ast'] / last_15['fg']
last_15['ast_perc_opp'] = last_15['ast_opp'] / last_15['fg_opp']

starters_15_games = last_15.dropna(axis=0).copy()
starters_15_games.tail()

target         fg  \
date       visitor                home                                        
2022-03-07 Houston Rockets        Miami Heat                 0.0  51.266667   
           Los Angeles Lakers     San Antonio Spurs          0.0  60.133333   
           New York Knicks        Sacramento Kings           0.0  58.133333   
           Portland Trail Blazers Minnesota Timberwolves     0.0  55.733333   
           Utah Jazz              Dallas Mavericks           0.0  55.533333   

                                                                 fga  \
date       visitor                home                                 
2022-03-07 Houston Rockets        Miami Heat              109.200000   
           Los Angeles Lakers     San Antonio Spurs       123.133333   
           New York Knicks        Sacramento Kings        122.800000   
           Portland Trail Blazers Minnesota Timberwolves  120.600000   
           Utah Jazz              Dallas Mavericks        114.266667   

                                                                 3p  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              17.400000   
           Los Angeles Lakers     San Antonio Spurs       14.666667   
           New York Knicks        Sacramento Kings        15.666667   
           Portland Trail Blazers Minnesota Timberwolves  17.666667   
           Utah Jazz              Dallas Mavericks        21.000000   

                                                                3pa    ft  \
date       visitor                home                                      
2022-03-07 Houston Rockets        Miami Heat              45.933333  24.6   
           Los Angeles Lakers     San Antonio Spurs       40.000000  21.4   
           New York Knicks        Sacramento Kings        43.933333  28.2   
           Portland Trail Blazers Minnesota Timberwolves  50.666667  24.8   
           Utah Jazz              Dallas Mavericks        51.666667  23.0   

                                                                fta  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              33.800000   
           Los Angeles Lakers     San Antonio Spurs       29.266667   
           New York Knicks        Sacramento Kings        38.333333   
           Portland Trail Blazers Minnesota Timberwolves  32.800000   
           Utah Jazz              Dallas Mavericks        29.866667   

                                                                orb  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              11.000000   
           Los Angeles Lakers     San Antonio Spurs       12.466667   
           New York Knicks        Sacramento Kings        14.866667   
           Portland Trail Blazers Minnesota Timberwolves  14.066667   
           Utah Jazz              Dallas Mavericks        12.333333   

                                                                drb  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              41.533333   
           Los Angeles Lakers     San Antonio Spurs       44.733333   
           New York Knicks        Sacramento Kings        44.533333   
           Portland Trail Blazers Minnesota Timberwolves  42.866667   
           Utah Jazz              Dallas Mavericks        42.800000   

                                                                trb  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              52.533333   
           Los Angeles Lakers     San Antonio Spurs       57.200000   
           New York Knicks        Sacramento Kings        59.400000   
           Portland Trail Blazers Minnesota Timberwolves  56.933333   
           Utah Jazz              Dallas 

## Correlations

In [43]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in starters_15_games:
    corr_p = pearsonr(starters_15_games['target'], starters_15_games[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print statistically significant correlations
starters_corr = corr_df[corr_df['p-value'] < .05].sort_values(['corr'], axis=0, ascending=False)
starters_corr

,corr,p-value,stat
0,1.00,0.0,target
4,0.71,0.0,3pa
113,0.71,0.0,3par
3,0.69,0.0,3p
21,0.68,0.0,3pa_opp
...,...,...,...
24,-0.29,0.0,orb_opp
115,-0.30,0.0,ftr
116,-0.37,0.0,ftr_opp
117,-0.38,0.0,orb_perc


## Bench Analysis

In [44]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = bench_df.copy()
X = bench_df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

### Last 15 Performances (Unweighted)

In [45]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']

last_15 = last_15_games.copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15[stat + '_trend'] = \
                            last_15[stat + '_trend'] + \
                            z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:10])

last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

# Calculate percentages
last_15['fg_perc'] = last_15['fg'] / last_15['fga']
last_15['fg_perc_opp'] = last_15['fg_opp'] / last_15['fga_opp']

last_15['3p_perc'] = last_15['3p'] / last_15['3pa']
last_15['3p_perc_opp'] = last_15['3p_opp'] / last_15['3pa_opp']

last_15['ft_perc'] = last_15['ft'] / last_15['fta']
last_15['ft_perc_opp'] = last_15['ft_opp'] / last_15['fta_opp']

# Calculate advanced stats
last_15['ts_perc'] = last_15['pts'] / (2 * (last_15['fga'] + .44 * last_15['fta']))
last_15['ts_perc_opp'] = last_15['pts_opp'] / (2 * (last_15['fga_opp'] + .44 * last_15['fta_opp']))

last_15['efg_perc'] = (last_15['fg'] + (.5 * last_15['3p'])) / last_15['fga']
last_15['efg_perc_opp'] = (last_15['fg_opp'] + (.5 * last_15['3p_opp'])) / last_15['fga_opp']

last_15['3par'] = last_15['3pa'] / last_15['fga']
last_15['3par_opp'] = last_15['3pa_opp'] / last_15['fga_opp']

last_15['ftr'] = last_15['fta'] / last_15['fga']
last_15['ftr_opp'] = last_15['fta_opp'] / last_15['fga_opp']

last_15['orb_perc'] = last_15['orb'] / (last_15['orb'] + last_15['drb_opp'])
last_15['orb_perc_opp'] = last_15['orb_opp'] / (last_15['orb_opp'] + last_15['drb'])

last_15['drb_perc'] = last_15['drb'] / (last_15['drb'] + last_15['orb_opp'])
last_15['drb_perc_opp'] = last_15['drb_opp'] / (last_15['drb_opp'] + last_15['orb'])

last_15['trb_perc'] = last_15['trb'] / (last_15['trb'] + last_15['trb_opp'])
last_15['trb_perc_opp'] = last_15['trb_opp'] / (last_15['trb_opp'] + last_15['trb'])

last_15['ast_perc'] = last_15['ast'] / last_15['fg']
last_15['ast_perc_opp'] = last_15['ast_opp'] / last_15['fg_opp']

bench_15_games = last_15.dropna(axis=0).copy()
bench_15_games.tail()

target         fg  \
date       visitor                home                                        
2022-03-07 Houston Rockets        Miami Heat                 0.0  27.533333   
           Los Angeles Lakers     San Antonio Spurs          0.0  25.800000   
           New York Knicks        Sacramento Kings           0.0  21.666667   
           Portland Trail Blazers Minnesota Timberwolves     0.0  26.666667   
           Utah Jazz              Dallas Mavericks           0.0  25.866667   

                                                                fga  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              61.600000   
           Los Angeles Lakers     San Antonio Spurs       57.066667   
           New York Knicks        Sacramento Kings        52.733333   
           Portland Trail Blazers Minnesota Timberwolves  57.866667   
           Utah Jazz              Dallas Mavericks        56.266667   

                                                                 3p  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat               9.133333   
           Los Angeles Lakers     San Antonio Spurs        8.733333   
           New York Knicks        Sacramento Kings         7.400000   
           Portland Trail Blazers Minnesota Timberwolves  10.600000   
           Utah Jazz              Dallas Mavericks         8.800000   

                                                                3pa  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              27.733333   
           Los Angeles Lakers     San Antonio Spurs       25.933333   
           New York Knicks        Sacramento Kings        24.400000   
           Portland Trail Blazers Minnesota Timberwolves  28.266667   
           Utah Jazz              Dallas Mavericks        26.333333   

                                                                 ft  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              11.866667   
           Los Angeles Lakers     San Antonio Spurs       10.866667   
           New York Knicks        Sacramento Kings        11.933333   
           Portland Trail Blazers Minnesota Timberwolves  10.200000   
           Utah Jazz              Dallas Mavericks         9.133333   

                                                                fta       orb  \
date       visitor                home                                          
2022-03-07 Houston Rockets        Miami Heat              15.733333  7.666667   
           Los Angeles Lakers     San Antonio Spurs       15.000000  7.600000   
           New York Knicks        Sacramento Kings        14.933333  6.600000   
           Portland Trail Blazers Minnesota Timberwolves  12.933333  5.933333   
           Utah Jazz              Dallas Mavericks        12.400000  8.400000   

                                                                drb  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              22.000000   
           Los Angeles Lakers     San Antonio Spurs       21.666667   
           New York Knicks        Sacramento Kings        23.733333   
           Portland Trail Blazers Minnesota Timberwolves  22.866667   
           Utah Jazz              Dallas Mavericks        22.600000   

                                                                trb  \
date       visitor                home                                
2022-03-07 Houston Rockets        Miami Heat              29.666667   
           Los Angeles Lakers     San Antonio Spurs       29.266667   
           New York Knicks        Sacramento Kings        30.333333   
           Portland Trail Blazers Minnesota Timberwolves  28.800000   
           Utah Jazz

## Correlations of Bench

In [46]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in bench_15_games:
    corr_p = pearsonr(bench_15_games['target'], bench_15_games[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print statistically significant correlations
bench_corr = corr_df[corr_df['p-value'] < .05].sort_values(['corr'], axis=0, ascending=False)
bench_corr

,corr,p-value,stat
0,1.00,0.0,target
21,0.65,0.0,3pa_opp
114,0.64,0.0,3par_opp
4,0.63,0.0,3pa
113,0.62,0.0,3par
...,...,...,...
51,-0.11,0.0,mp_std
118,-0.29,0.0,orb_perc_opp
115,-0.34,0.0,ftr
117,-0.35,0.0,orb_perc


# Comparison of Starters to Bench

In [47]:
corr_df = pd.merge(starters_corr.drop(['p-value'], axis=1), 
                   bench_corr.drop(['p-value'], axis=1),
                   left_on=['stat'], right_on=['stat'],
                   how='outer',
                   suffixes=['_starter', '_bench'])
corr_df.sort_values(['stat'], axis=0)

,corr_starter,stat,corr_bench
3,0.69,3p,0.60
6,0.66,3p_opp,0.62
10,0.44,3p_opp_std,0.38
38,0.12,3p_perc,0.04
47,0.04,3p_perc_opp,-0.02
...,...,...,...
58,-0.02,trb_perc,-0.05
55,0.02,trb_perc_opp,0.05
59,-0.02,trb_std,0.06
11,0.41,ts_perc,0.36


## Save dataframe with significantly correlated stats

In [48]:
correlation = .6

starter_stats = starters_corr[starters_corr['corr'].abs() >= correlation]['stat']
starters_df = starters_15_games[starter_stats]

bench_stats = bench_corr[bench_corr['corr'].abs() >= correlation]['stat']
bench_df = bench_15_games[bench_stats]

df = pd.merge(starters_df, bench_df, 
              left_on=['date', 'visitor', 'home'], 
              right_on=['date', 'visitor', 'home'], 
              how='outer', suffixes=['_starters', '_bench'])

df = df.drop(['target_bench', 'target_starters'], axis=1)

df.to_csv('backend/data/inputs/3p/game_details.csv')